In [2]:
import redis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cx_Oracle
import pickle


In [3]:
# połączenie z bazą Redis
r = redis.Redis(host='localhost', port=6379, db=0)


In [4]:
# połączenie z bazą Oracle
connection = cx_Oracle.connect(user="s108261", password="s108261",
                               dsn="217.173.198.135:1521/tpdb")
cursor = connection.cursor()
cursor.execute("ALTER SESSION SET nls_date_format='YYYY-MM-DD'")

In [44]:
factory = 'Hybrid Oak'
dateS = '2020-11-30'
dateT = '2023-11-30'

name = "Karol"
surname = "Strognell"

1 przykład - optymalizacja wyciągania danych ze Stocku.

In [15]:
def getOracleStock(factory, dateS, dateT):
    querry = f"select * from STOCK INNER JOIN Factories on STOCK.factory_ID = Factories.factory_ID " \
           + f"where name = '{factory}' and PRODUCTION_DATE < '{dateT}' and " \
           + f"PRODUCTION_DATE > '{dateS}'"
    data = pd.read_sql(querry,connection)
    data = pickle.dumps(data)
    #dump
    return data

def sendToCache(index, data):
    try:
        r.set(index, data)
        print("success")
        return 0
    except:
        print("failed")
        return 1

def getStockData(factory, dateS, dateT):
    index = (factory+":"+dateS+"-"+dateT)
    data = r.get(index)
    if data:
        print("Taken from Cache")
        #dump
        data = pickle.loads(data)
        return data

    else:
        data = getOracleStock(factory, dateS, dateT)
        sendToCache(index, data)
        print("Taken from Ocarle DB, pushed to cache")
        #dump
        data = pickle.loads(data)
        return data
    
    
    
    

In [22]:
newData = getStockData(factory, dateS, dateT)
print(type(newData))
print(newData)

success
Taken from Ocarle DB, pushed to cache
<class 'pandas.core.frame.DataFrame'>
   SERIALNUMBER_ID       SERIALNUMBER PRODUCTION_DATE  FACTORY_ID  SKU_ID  \
0              653  JN8AF5MR7CT853222      2023-04-26           2      74   
1              273  WBA3F9C56DF392859      2023-08-20           2      71   
2              306  WAUJC58E33A414233      2023-03-11           2      49   

   S_MEASURE_ID  FACTORY_ID             NAME COUNTRY POSTAL_CODE       CITY  
0           351           2  Blunt Spikerush  Poland      82-230  Nowy Staw  
1           409           2  Blunt Spikerush  Poland      82-230  Nowy Staw  
2           235           2  Blunt Spikerush  Poland      82-230  Nowy Staw  


2 przykład - wyciąganie danych z tabeli Employees - stanowisko

In [23]:
def getOracleEmpPos(name, surname):
    querry = f"SELECT employees.name, employees.surname, employees.position, factories.name AS factoryName FROM EMPLOYEES" \
         f" INNER JOIN Factories on EMPLOYEES.factory_ID = Factories.factory_ID" \
         f" WHERE employees.name = '{name}' AND employees.surname = '{surname}'"
    data = pd.read_sql(querry,connection)
    data = pickle.dumps(data)
    return data

def getEmpPosData(name, surname):
    index = (name+":"+surname)
    data = r.get(index)
    if data:
        print("Taken from Cache")
        data = pickle.loads(data)
        return data

    else:
        data = getOracleEmpPos(name, surname)
        sendToCache(index, data)
        print("Taken from Ocarle DB, pushed to cache")
        data = pickle.loads(data)
        return data



In [29]:
newData = getEmpPosData(name,surname)
print(type(newData))
print(newData)

Taken from Cache
<class 'pandas.core.frame.DataFrame'>
    NAME    SURNAME  POSITION          FACTORYNAME
0  Karol  Strognell  security  Woodland Islandfern


3 przykład - wyciągany sprzedaży dla fabryk

In [37]:
def getOracleSales(factory):
    querry = f"SELECT AVG(INCOME), SALES.YEAR, Factories.name AS FactoryName FROM SALES" \
            f" INNER JOIN Factories on SALES.factory_ID = Factories.factory_ID" \
            f" WHERE Factories.name = '{factory}'" \
            f" GROUP BY Factories.name, SALES.YEAR"
    data = pd.read_sql(querry,connection)
    data = pickle.dumps(data)
    return data

def getSalesData(factory):
    index = (factory)
    data = r.get(index)
    if data:
        print("Taken from Cache")
        data = pickle.loads(data)
        return data

    else:
        data = getOracleSales(factory)
        sendToCache(index, data)
        print("Taken from Ocarle DB, pushed to cache")
        data = pickle.loads(data)
        return data

In [49]:
nes = getSalesData(factory)
nes

Taken from Cache


,AVG(INCOME),YEAR,FACTORYNAME
0,2063.06,2021,Hybrid Oak
1,4490.29,2022,Hybrid Oak
2,5116.78,2020,Hybrid Oak


In [50]:
r.flushdb()

True